### Load Excel Files

This notebook holds the source code for the resuable function to load an excel file into a Snowflake table.  It should be deployed to your project via deploy_notebooks.sql

In [ ]:
# Imports

import sys
import pandas as pd
from openpyxl import load_workbook


In [ ]:
# define load_excel_file

def load_excel_file(session, params):

    source_schema = params['source_schema']
    source_table = params['source_table']
    source_directory = params.get('source_directory', '')
    source_file = params['source_file']
    source_worksheet = params['source_worksheet']
    source_path = f"{source_schema}/{source_table}/{source_directory}/{source_file}"

    local_directory = "./"
    local_path = f"{local_directory}{source_file}"

    target_schema = params['target_schema']
    target_table = params['target_table']
    
    # Copy file to local
    session.file.get(source_path, local_directory)

    # Open local and load workbook
    with open(local_path, 'rb') as f:
        workbook = load_workbook(f)
        sheet = workbook[source_worksheet]
        data = sheet.values

        # Set the first row as the header
        columns = next(data)[0:]
        
        # Create a pandas DataFrame with the remaining rows
        pandas_df = pd.DataFrame(data, columns=columns)
    
        # Convert to a snowpark DataFrame
        snowpark_df = session.create_dataframe(pandas_df)

        # Load into table
        session.use_schema(target_schema)
        snowpark_df.write.mode("overwrite").save_as_table(target_table)

In [ ]:
# Get parameters
session = sys.argv[0]
params = sys.argv[1]

# Load file
load_excel_file(session, params)